In [2]:
import openpyxl as op
import urllib.request as req
import xml.etree.ElementTree as ET
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import os
from pprint import pprint

DATA_PATH = "../data"
XML_SCHEMAS = {
    'R2.0.9.S03.E01' : "{http://publications.europa.eu/resource/schema/ted/R2.0.9/publication}",
    'R2.0.9.S04.E01' : "{http://publications.europa.eu/resource/schema/ted/R2.0.9/publication}",
    'R2.0.9.S02.E01' : "{ted/R2.0.9.S02/publication}",
    'R2.0.9.S01.E01' : "{http://formex.publications.europa.eu/ted/schema/export/R2.0.9.S01.E01}",
    None : "{http://publications.europa.eu/TED_schema/Export}"
}
LOG_MODE = False

### Downloading contracts

NOTE: This isn't neccessary anymore. There is a different way to download all frontex tender award notices in bulk from the TED website. See notes doc.

In [ ]:
# download all frontex contracts referenced in sheets
# doc_history = set()
# contract_counter = 0
# for sheet_file in os.listdir(DATA_PATH + "/sheets/"):
#     wb = op.load_workbook(DATA_PATH + "/sheets/" + sheet_file)
#     sheet = wb.active
    
#     # download all contracts in XML format
#     for row in sheet.rows:
#         if "frontex" in row[3].value.lower() or "border" in row[3].value.lower():
#             url = row[0].hyperlink.target.replace(":TEXT:", ":XML:")
#             doc_id = url.split(":")[3]
            
#             if doc_id not in doc_history:
#                 # add document to history
#                 doc_history.add(doc_id)
            
#                 # download document
#                 try:
#                     req.urlretrieve(url, DATA_PATH + "/XML/" + doc_id + ".xml")
#                     if LOG_MODE: print(f"Saved {row[2].value} to {doc_id}.xml")
#                 except Exception as E:
#                     print(E)

#                 contract_counter += 1

### Extracting data from XML documents

In [ ]:
# Lets explore a contract!
contract = "309706-2014.xml"
xml_tree = ET.parse(DATA_PATH + "/XML/" + contract)
schema = XML_SCHEMAS[xml_tree.getroot().attrib.get('VERSION')]
FORM_SECTION = xml_tree.getroot().find(schema + "FORM_SECTION")[2][0]
OBJECT_CONTRACT = FORM_SECTION.find(schema + "OBJECT_CONTRACT_INFORMATION_CONTRACT_AWARD_NOTICE")
OBJECT_DESCR = OBJECT_CONTRACT.find(schema + "DESCRIPTION_AWARD_NOTICE_INFORMATION")
AWARD_OF_CONTRACT = FORM_SECTION.findall(schema + "AWARD_OF_CONTRACT")
COSTS_RANGE_AND_CURRENCY_WITH_VAT_RATE = AWARD_OF_CONTRACT[1].find(schema + "CONTRACT_VALUE_INFORMATION").find(schema + "COSTS_RANGE_AND_CURRENCY_WITH_VAT_RATE")
VALUE_COST = COSTS_RANGE_AND_CURRENCY_WITH_VAT_RATE.find(schema + "VALUE_COST")

temp = AWARD_OF_CONTRACT[0].find(schema + "ECONOMIC_OPERATOR_NAME_ADDRESS").find(schema + "CONTACT_DATA_WITHOUT_RESPONSIBLE_NAME").find(schema + "ORGANISATION").find(schema + "OFFICIALNAME").text
pprint(temp)
pprint(list(temp))

In [3]:
data = pd.DataFrame(columns=["Contract", "Title", "Type of contract", "Short description", "Total value", "Place of performance", "Contractor"])

# Reasons for skipping:
#     - different spellings of Lot no, TODO: handle these exceptions
#     - Different overall form structure
error_counter = {
    "Cannot find FORM_SECTION" : 0,
    "Cannot find OBJECT_CONTRACT" : 0,
    "Cannot find LOT_NO in award section": 0,
    "Missing place of performance" : 0,
    "Missing value" : 0,
    "Missing contractor" : 0
}
contract_counter, fail_counter = 0, 0
skip = ['185944-2018.xml', '245168-2020.xml', '509257-2018.xml', '019628-2016.xml', '412856-2017.xml']
for contract in os.listdir(DATA_PATH + "/XML/"):
    if contract in skip:
        fail_counter += 1
        continue
        
    if LOG_MODE: print(f"Extracting data: {contract}")
    
    # parse the XML structure
    xml_tree = ET.parse(DATA_PATH + "/XML/" + contract)
    VERSION = xml_tree.getroot().attrib.get('VERSION')
    old_version = VERSION is None
    schema = XML_SCHEMAS[VERSION]
    
    # the english translation of the form
    FORM_SECTION = xml_tree.getroot().find(schema + "FORM_SECTION")
    if FORM_SECTION is None:
        if LOG_MODE: print("    Cannot find FORM_SECTION")
        error_counter["Cannot find FORM_SECTION"] += 1
        fail_counter += 1
        continue
        
    if old_version:
        FORM_SECTION = FORM_SECTION[2][0]
    else:
        FORM_SECTION = FORM_SECTION[2]        

    # the object description section(s)
    query = "OBJECT_CONTRACT_INFORMATION_CONTRACT_AWARD_NOTICE" if old_version else "OBJECT_CONTRACT"
    OBJECT_CONTRACT = FORM_SECTION.find(schema + query)
    if OBJECT_CONTRACT is None:
        if LOG_MODE: print("    Cannot find OBJECT_CONTRACT")
        error_counter["Cannot find OBJECT_CONTRACT"] += 1
        fail_counter += 1
        continue
        
    query = "DESCRIPTION_AWARD_NOTICE_INFORMATION" if old_version else "OBJECT_DESCR"
    OBJECT_DESCR = OBJECT_CONTRACT.findall(schema + query) 
    
    # find out if divided into lots and if so save multiple object descriptions
    # old version are never divided into lots
    has_lots = not old_version and OBJECT_CONTRACT.find(schema + "LOT_DIVISION") is not None
    if has_lots:
        lot_dict = {int(desc.find(schema + "LOT_NO").text) : desc for desc in OBJECT_DESCR}
       
    # get the type of contract (which is independend of awards or lots)
    if old_version:
        type_of_contract = OBJECT_DESCR[0].find(schema + "TYPE_CONTRACT_LOCATION_W_PUB").find(schema + "TYPE_CONTRACT").attrib.get('VALUE')
    else:
        type_of_contract = OBJECT_CONTRACT.find(schema + "TYPE_CONTRACT").attrib["CTYPE"]

    # there might be multiple contract awards
    query = "AWARD_OF_CONTRACT" if old_version else "AWARD_CONTRACT"
    awards = FORM_SECTION.findall(schema + query)
    for award in awards:
        # relevant object description depend on wether contract is divided into lots or not
        if has_lots:
            LOT_NO = award.find(schema + "LOT_NO")
            if LOT_NO is not None:
                LOT_NO = int(LOT_NO.text)
            else:
                if LOG_MODE: print("    Cannot find LOT_NO in award section")
                error_counter["Cannot find LOT_NO in award section"] += 1
                fail_counter += 1
                break
            
            # object description might be missing for a certain lot
            if LOT_NO in lot_dict:
                current_obj_desc = lot_dict[LOT_NO]
            else:
                current_obj_desc = None
        else:
            current_obj_desc = OBJECT_DESCR[0]
        
        # get the contract title
        query = "CONTRACT_TITLE" if old_version else "TITLE"
        title = award.find(schema + query)
        # sometimes title missing in the award of contract
        if title is None:
            if has_lots:
                title = current_obj_desc.find(schema + "TITLE")[0].text
            else:
                # if there are no lots, the title is the general contract title
                title = current_obj_desc.find(schema + "TITLE_CONTRACT")[0].text if old_version else OBJECT_CONTRACT.find(schema + "TITLE")[0].text
        else:
            title = title[0].text
        
        # short description and p.o.p. depend on an object description in the same contract
        # which might be missing when contract is not awarded
        if current_obj_desc is not None:
            # get the description
            query = "SHORT_CONTRACT_DESCRIPTION" if old_version else "SHORT_DESCR"
            short_descr = " ".join([x.text for x in current_obj_desc.find(schema + query)])
            
            # get the place of performance NUTS code
            if old_version:
                # either location in general or the NUTS code specifically might be missing
                LOCATION_NUTS = current_obj_desc.find(schema + "LOCATION_NUTS")
                if LOCATION_NUTS is None:
                    if LOG_MODE: print("    Missing place of performance")
                    error_counter["Missing place of performance"] += 1
                    NUTS = None
                else:
                    NUTS = LOCATION_NUTS.find(schema + "NUTS")
                
                if NUTS is None:
                    if LOG_MODE: print("    Missing place of performance")
                    error_counter["Missing place of performance"] += 1
                    place_of_performance = None
                else:
                    place_of_performance = NUTS.attrib.get('CODE')
            else:
                place_of_performance = [elem for elem in list(current_obj_desc) if elem.tag[-4:] == "NUTS"][0].attrib["CODE"]
        else:
            short_descr = None
            place_of_performance = None
        
        if old_version:
            # get the value
            info = award.find(schema + "CONTRACT_VALUE_INFORMATION")
            costs_range = info.find(schema + "COSTS_RANGE_AND_CURRENCY_WITH_VAT_RATE") if info is not None else None         
            value_cost = costs_range.find(schema + "VALUE_COST") if costs_range is not None and costs_range.attrib.get("CURRENCY") == "EUR" else None

            if value_cost is not None:
                value = float(value_cost.text.replace(" ", "").replace(",", "."))
            else:
                value = None
                if LOG_MODE: print("    Missing value")
                error_counter["Missing value"] += 1
            
            # get the contractors
            name_address = award.find(schema + "ECONOMIC_OPERATOR_NAME_ADDRESS")
            contact = name_address.find(schema + "CONTACT_DATA_WITHOUT_RESPONSIBLE_NAME") if name_address is not None else None
            organisation = contact.find(schema + "ORGANISATION") if contact is not None else None
            official = organisation.find(schema + "OFFICIALNAME") if organisation is not None else None
            
            if official is not None and official.text != "No offers received":
                contractor_names = [official.text]
            else:
                value = None
                if LOG_MODE: print("    Missing contractor")
                error_counter["Missing contractor"] += 1
        else:
            # value and contractors depend on wether the contract is awarded or not        
            is_awarded = award.find(schema + "NO_AWARDED_CONTRACT") is None
            if is_awarded:
                # get the value of award
                value = award.find(schema + "AWARDED_CONTRACT").find(schema + "VAL_TOTAL")
                if value is None:
                    value = award.find(schema + "AWARDED_CONTRACT").find(schema + "VALUES").find(schema + "VAL_TOTAL")
                value = float(value.text)

                # get the contractors of award
                contractor_names = []
                contractors = award.find(schema + "AWARDED_CONTRACT").findall(schema + "CONTRACTOR")
                if len(contractors) == 0:
                    contractors = award.find(schema + "AWARDED_CONTRACT").find(schema + "CONTRACTORS").findall(schema + "CONTRACTOR")

                for contractor in contractors:
                    contractor_names.append(contractor.find(schema + "ADDRESS_CONTRACTOR").find(schema + "OFFICIALNAME").text)
            else:
                value = None
                contractor_names = None


        data = data.append({
            "Contract" : contract[:-4],
            "Title": title,
            "Type of contract" : type_of_contract,
            "Short description" : short_descr,
            "Total value" : value,
            "Place of performance" : place_of_performance,
            "Contractor" : contractor_names
        }, ignore_index=True)
    contract_counter += 1

print(f"{contract_counter} succesfully contracts processed, {fail_counter} failed, {sum(error_counter.values())} errors")

# note some of these aren't errors per se, for example on older versions of the documents
# it isn't explicitely stated if a contract is awarded, so a missing value and missing contractors
# might just mean a contract not awarded. Older versions also simply don't list value on a per
# award basis only a total value without information on how this is distributed across the awards.
print("Error counts: ")
pprint(error_counter)

158 succesfully contracts processed, 8 failed, 78 errors
Error counts: 
{'Cannot find FORM_SECTION': 0,
 'Cannot find LOT_NO in award section': 1,
 'Cannot find OBJECT_CONTRACT': 2,
 'Missing contractor': 3,
 'Missing place of performance': 33,
 'Missing value': 39}


In [4]:
# quick description of the dataset
print(f"Number of rows: {len(data)}")
print("Missing values per column: ")
for column in data.columns:
    n_missing_values = sum(data[column].isna())
    percent = n_missing_values / len(data[column]) * 100
    print(f"    {column}: \t{n_missing_values} ({percent:.2f}%)")
data.sample(5)

Number of rows: 287
Missing values per column: 
    Contract: 	0 (0.00%)
    Title: 	0 (0.00%)
    Type of contract: 	0 (0.00%)
    Short description: 	1 (0.35%)
    Total value: 	87 (30.31%)
    Place of performance: 	27 (9.41%)
    Contractor: 	47 (16.38%)


,Contract,Title,Type of contract,Short description,Total value,Place of performance,Contractor
35,054220-2019,Purchase of the Transportation Network Spatial...,SERVICES,Purchase of the transportation network spatial...,NaN,00,None
13,022822-2019,International Export/Import Courier Services,SERVICES,International export/import courier services.,3040000.0,00,[TNT Express Worldwide (Poland) sp. z o.o.]
1,000007-2017,Framework contract for the provision of passen...,SERVICES,Provision of passenger services by bus in Gree...,150000.0,GR,[Vagia Tzislaki — Lesvorama General Travel Age...
134,236248-2020,Shoes and Accessories,SUPPLIES,"Items under Lot 1 are: Winter jacket (unisex),...",500000.0,PL911,[Robert Wagner — Wagner Werbemittel und Textil...
207,387511-2015,Framework contract for the provision of interi...,SERVICES,Frontex has concluded a framework contract wit...,1500000.0,PL127,[Grafton Recruitment Polska Sp. z o.o.]


In [6]:
# save dataframe
data.to_pickle(DATA_PATH + "/processed/dataset.pkl")